## Model Deployment

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sqlite3
from glob import glob
import joblib
import pandas as pd
import requests
from arch.univariate.base import ARCHModelResult
from config import settings
from data import SQLRepository
from model import GarchModel

In [2]:
#Open a connection to a SQL database using sqlite3.
connection = sqlite3.connect(settings.db_name, check_same_thread = False)
repo = SQLRepository(connection = connection)

print("repo type:", type(repo))
print("repo.connection type:", type(repo.connection))

repo type: <class 'data.SQLRepository'>
repo.connection type: <class 'sqlite3.Connection'>


In [3]:
# Instantiate a `GarchModel`
gm_ambuja = GarchModel(ticker="AMBUJACEM.BSE", repo=repo, use_new_data=False)

# Does `gm_ambuja` have the correct attributes?
assert gm_ambuja.ticker == "AMBUJACEM.BSE"
assert gm_ambuja.repo == repo
assert not gm_ambuja.use_new_data
assert gm_ambuja.model_directory == settings.model_directory

In [4]:
# Instantiate `GarchModel`, use new data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=True)

# Check that model doesn't have `data` attribute yet
assert not hasattr(model_shop, "data")

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Does model now have `data` attribute?
assert hasattr(model_shop, "data")

# Is the `data` a Series?
assert isinstance(model_shop.data, pd.Series)

# Is Series correct shape?
assert model_shop.data.shape == (1000,)

model_shop.data.head()

date
2021-03-23   -0.633628
2021-03-24   -3.097244
2021-03-25   -0.587544
2021-03-26    0.378251
2021-03-30    0.730099
Name: return, dtype: float64

In [5]:
# Instantiate `GarchModel`, use old data
model_shop = GarchModel(ticker="SHOPERSTOP.BSE", repo=repo, use_new_data=False)

# Wrangle data
model_shop.wrangle_data(n_observations=1000)

# Fit GARCH(1,1) model to data
model_shop.fit(p=1, q=1)

# Does `model_shop` have a `model` attribute now?
assert hasattr(model_shop, "model")

# Is model correct data type?
assert isinstance(model_shop.model, ARCHModelResult)

# Does model have correct parameters?
assert model_shop.model.params.index.tolist() == ["mu", "omega", "alpha[1]", "beta[1]"]

# Check model parameters
model_shop.model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2317.60
Distribution:                  Normal   AIC:                           4643.19
Method:            Maximum Likelihood   BIC:                           4662.82
                                        No. Observations:                 1000
Date:                Tue, Apr 01 2025   Df Residuals:                      999
Time:                        03:30:15   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.1036  7.621e-02      1.359      0.174 [-4.578e-02,  0.253]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0461  3.523e-02      1.309      0.190  [-2.293e-02,  0.115]
alpha[1]       0.0103  3.897e-03      2.635  8.407e-03 [2.632e-03,1.791e-02]
beta[1]        0.9820  4.140e-03    237.187      0.000     [  0.974,  0.990]
============================================================================

Covariance estimator: robust
"""

In [6]:
# Generate prediction from `model_shop`
prediction = model_shop.predict_volatility(horizon=5)

# Is prediction a dictionary?
assert isinstance(prediction, dict)

# Are keys correct data type?
assert all(isinstance(k, str) for k in prediction.keys())

# Are values correct data type?
assert all(isinstance(v, float) for v in prediction.values())

In [7]:
filename = model_shop.dump()
filename

Saving model to: models\2025-04-01T03-30-16-116302_SHOPERSTOP_BSE.pkl


'models\\2025-04-01T03-30-16-116302_SHOPERSTOP_BSE.pkl'

In [8]:
import re
def load(ticker):
    """Load most recent model in `model_directory` for the given ticker.
    
    Parameters
    ----------
    ticker : str
        The ticker symbol to load (e.g., "SHOPERSTOP.BSE")
    model_directory : str
        Path to directory containing saved models
        
    Returns
    -------
    object
        The loaded model, or None if loading failed
    """
    try:
        # Sanitize the ticker to match how it was saved
        sanitized_ticker = re.sub(r'[<>:"/\\|?*]', '_', ticker).replace('.', '_')
        
        # Create pattern for glob search (match any timestamp + our ticker)
        pattern = os.path.join(settings.model_directory, f"*_{sanitized_ticker}.pkl")
        
        # Get all matching files
        matching_files = glob(pattern)
        
        if not matching_files:
            print(f"No model files found for ticker: {ticker}")
            return None
            
        # Sort files by modification time (newest first)
        matching_files.sort(key=os.path.getmtime, reverse=True)
        
        # Get the most recent file
        latest_file = matching_files[0]
        
        # Load the model
        model = joblib.load(latest_file)
        print(f"Successfully loaded model from: {latest_file}")
        return model
        
    except IndexError:
        print(f"No valid model files found for pattern: {pattern}")
    except FileNotFoundError:
        print(f"Model directory not found: {model_directory}")
    except Exception as e:
        print(f"Error loading model for {ticker}: {str(e)}")
    
    return None

# Example usage
model_shop = load(ticker="SHOPERSTOP.BSE")

Successfully loaded model from: models\2025-04-01T03-30-16-116302_SHOPERSTOP_BSE.pkl


In [9]:
model_shop = load(ticker="SHOPERSTOP.BSE")

Successfully loaded model from: models\2025-04-01T03-30-16-116302_SHOPERSTOP_BSE.pkl


In [10]:
model_shop.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                 return   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -2317.60
Distribution:                  Normal   AIC:                           4643.19
Method:            Maximum Likelihood   BIC:                           4662.82
                                        No. Observations:                 1000
Date:                Tue, Apr 01 2025   Df Residuals:                      999
Time:                        03:30:15   Df Model:                            1
                                 Mean Model                                
===========================================================================
                 coef    std err          t      P>|t|     95.0% Conf. Int.
---------------------------------------------------------------------------
mu             0.1036  7.621e-02      1.359      0.174 [-4.578e-02,  0.253]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0461  3.523e-02      1.309      0.190  [-2.293e-02,  0.115]
alpha[1]       0.0103  3.897e-03      2.635  8.407e-03 [2.632e-03,1.791e-02]
beta[1]        0.9820  4.140e-03    237.187      0.000     [  0.974,  0.990]
============================================================================

Covariance estimator: robust
"""

## Main Module

In [11]:
url = "http://localhost:8008/hello"
response = requests.get(url=url)

print("response code:", response.status_code)
response.json()

response code: 200


{'message': 'Hello world'}

In [12]:
from main import FitIn, FitOut

# Instantiate `FitIn`. Play with parameters.
fi = FitIn(
    ticker = "SHOPERSTOP.BSE",
    use_new_data = True,
    n_observations = 2000,
    p = 1,
    q = 1
)
print(fi)

# Instantiate `FitOut`. Play with parameters.
fo = FitOut(
   ticker = "SHOPERSTOP.BSE",
    use_new_data = True,
    n_observations = 2000,
    p = 1,
    q = 1,
    success = True,
    message = "model is ready to rock"
)
print(fo)

ticker='SHOPERSTOP.BSE' n_observations=2000 use_new_data=True p=1 q=1
ticker='SHOPERSTOP.BSE' n_observations=2000 use_new_data=True p=1 q=1 success=True message='model is ready to rock'


In [13]:
# URL of `/fit` path
url = "http://localhost:8008/fit"

# Data to send to path
json = {
    "ticker": "SHOPERSTOP.BSE",
    "use_new_data": False,
    "n_observations": 2000,
    "p": 1,
    "q": 1
}
# Response of post request
response = requests.post(url=url, json=json)
# Inspect response
print("response code:", response.status_code)
response.json()

response code: 200


{'ticker': 'SHOPERSTOP.BSE',
 'n_observations': 2000,
 'use_new_data': False,
 'p': 1,
 'q': 1,
 'success': True,
 'message': "Trained and saved 'models\\2025-04-01T03-30-28-476322_SHOPERSTOP_BSE.pkl'."}

In [14]:
from main import PredictIn, PredictOut

pi = PredictIn(ticker="SHOPERSTOP.BSE", n_days=5, use_new_data = False)
print(pi)

po = PredictOut(
    ticker="SHOPERSTOP.BSE", n_days=5, success=True, forecast={}, message="success", use_new_data = False
)
print(po)

n_days=5 ticker='SHOPERSTOP.BSE' use_new_data=False
n_days=5 ticker='SHOPERSTOP.BSE' use_new_data=False forecast={} success=True message='success'


In [18]:
# URL of `/predict` path
url = "http://localhost:8000/predict"
# Data to send to path
json = {
    "n_days": 5, 
    "ticker": "SHOPERSTOP.BSE",
    "use_new_data": False       
}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
response.json()

{'n_days': 5,
 'ticker': 'SHOPERSTOP.BSE',
 'use_new_data': False,
 'forecast': {'2025-03-31T00:00:00': 2.892467271747071,
  '2025-04-01T00:00:00': 2.8842176757326237,
  '2025-04-02T00:00:00': 2.876216748523103,
  '2025-04-03T00:00:00': 2.868457595184337,
  '2025-04-04T00:00:00': 2.8609334793275147},
 'success': True,
 'message': ''}

In [19]:
url = "http://localhost:8000/predict"
# Data to send to path
json = {
    "n_days": 5, 
    "ticker": "IBM",
    "use_new_data": True       
}
# Response of post request
response = requests.post(url=url, json=json)
# Response JSON to be submitted to grader
response.json()

{'n_days': 5,
 'ticker': 'IBM',
 'use_new_data': True,
 'forecast': {},
 'success': False,
 'message': "'GarchModel' object has no attribute 'model'"}

In [2]:
import requests
url = "http://localhost:8000/check_ticker"
json = {"ticker": "tesla"}
response = requests.post(url=url, json = json)
response.json()

{'ticker': 'tesla',
 'result': {'bestMatches': [{'1. symbol': 'TSLA',
    '2. name': 'Tesla Inc',
    '3. type': 'Equity',
    '4. region': 'United States',
    '5. marketOpen': '09:30',
    '6. marketClose': '16:00',
    '7. timezone': 'UTC-04',
    '8. currency': 'USD',
    '9. matchScore': '0.8889'},
   {'1. symbol': 'TL0.DEX',
    '2. name': 'Tesla Inc',
    '3. type': 'Equity',
    '4. region': 'XETRA',
    '5. marketOpen': '08:00',
    '6. marketClose': '20:00',
    '7. timezone': 'UTC+02',
    '8. currency': 'EUR',
    '9. matchScore': '0.7143'},
   {'1. symbol': 'TL0.FRK',
    '2. name': 'Tesla Inc',
    '3. type': 'Equity',
    '4. region': 'Frankfurt',
    '5. marketOpen': '08:00',
    '6. marketClose': '20:00',
    '7. timezone': 'UTC+02',
    '8. currency': 'EUR',
    '9. matchScore': '0.7143'},
   {'1. symbol': 'TSLA34.SAO',
    '2. name': 'Tesla Inc',
    '3. type': 'Equity',
    '4. region': 'Brazil/Sao Paolo',
    '5. marketOpen': '10:00',
    '6. marketClose': '17:30',